# IG Metall Bayern Gehaltsrechner
Stand: Tarifrunde 2025 (gültig von 1.4.2025 - 31.3.2026)

Basierend auf https://bayern.igmetall.de/aktuell/metall--und-elektroindustrie

In [ ]:
from dataclasses import dataclass

CONFIG_9b_2025 = {
    "entgeltgruppe": "9b",
    "grundentgelt": 4841.00,         # Monatsgrundentgelt laut Tarifabelle
    "leistungszulage_fix": 271.10,   # Fixe Leistungszulage in € (bis zu 28% des grundentgelts, im Durchschnitt müssen 14% erreicht werden können)
    "urlaubsgeld_faktor": 0.7,       # Ca. 70% eines Monatseinkommens
    "weihnachtsgeld_faktor": 0.55,   # 55% eines Monatseinkommens
    "t_zug_a_faktor": 0.275,         # 27,5% eines Monatseinkommens (T-ZUG A im Juli)
    "t_zug_b": 630,                  # Festbetrag für 2025 (T-ZUG B im Februar)
    "t_geld_faktor": 0.184,          # 18,4% eines Monatseinkommens (T-Geld im Juli)
    "t_geld_aktiv": False,           # Bei Nettoumsatzrendite unter 2,3% kann der Arbeitgeber das T-Geld aussetzen
    "abgaben_standard": 0.38,        # Annamhme von ca 38% Steuern und Sozialabgaben
    "abgaben_sonderzahlungen": 0.395 # Progressiver Steuersatz führt zu höheren Abgaben bei Sonderzahlungen
}

# Gemeinsame Werte für alle Tarifgruppen
DEFAULTS = {
    "leistungszulage_fix": 271.10,       # Fixe Leistungszulage in € (bis zu 28% des grundentgelts, im Durchschnitt müssen 14% erreicht werden können)
    "urlaubsgeld_faktor": 0.7,           # Ca. 70% eines Monatseinkommens
    "weihnachtsgeld_faktor": 0.55,       # 55% eines Monatseinkommens
    "t_zug_a_faktor": 0.275,             # 27,5% eines Monatseinkommens (T-ZUG A im Juli)
    "t_zug_b": 630,                      # Festbetrag für 2025 (T-ZUG B im Februar)
    "t_geld_faktor": 0.184,              # 18,4% eines Monatseinkommens (T-Geld im Juli)
    "t_geld_aktiv": False,               # Bei Nettoumsatzrendite unter 2,3% kann der Arbeitgeber das T-Geld aussetzen
    "abgaben_standard": 0.38,            # Annahme von ca. 38% Steuern und Sozialabgaben
    "abgaben_sonderzahlungen": 0.395     # Progressiver Steuersatz führt zu höheren Abgaben bei Sonderzahlungen
}

class TarifInfo:
    entgeltgruppe: str
    grundentgelt: float
    jahr: int
    leistungszulage_fix: float = 271.10
    urlaubsgeld_faktor: float = 0.7
    weihnachtsgeld_faktor: float = 0.55
    t_zug_a_faktor: float = 0.275
    t_zug_b: float = 630
    t_geld_faktor: float = 0.184
    t_geld_aktiv: bool = False
    abgaben_standard: float = 0.38
    abgaben_sonderzahlungen: float  = 0.395


TARIFGRUPPEN = [
    {"entgeltgruppe": "EG09 B", "grundentgelt": 4841, "jahr": 2025},
    {"entgeltgruppe": "EG10 A", "grundentgelt": 5103, "jahr": 2025},
    {"entgeltgruppe": "EG10 B", "grundentgelt": 5362, "jahr": 2025},
    {"entgeltgruppe": "EG11 B", "grundentgelt": 5362, "jahr": 2025},
    {"entgeltgruppe": "EG09 B", "grundentgelt": 4991, "jahr": 2026},
    {"entgeltgruppe": "EG10 A", "grundentgelt": 5261, "jahr": 2026},
    {"entgeltgruppe": "EG10 B", "grundentgelt": 5528, "jahr": 2026}
]


@dataclass
class TarifInfo:
    entgeltgruppe: str
    grundentgelt: float
    leistungszulage_fix: float = 271.10
    t_zug_b: float = 630
    urlaubsgeld_faktor: float = 0.7
    weihnachtsgeld_faktor: float = 0.55
    t_zug_a_faktor: float = 0.275
    t_geld_faktor: float = 0.184
    t_geld_aktiv: bool = False
    abgaben_standard: float = 0.38
    abgaben_sonderzahlungen: float  = 0.395

def berechne_jahresgehalt(info: TarifInfo):
    """
    Berechnet das Jahresbruttogehalt eines IG-Metall-Mitarbeiters in Bayern
    basierend auf Tarifgruppe, fixer Leistungszulage und Zuschlägen.
    """
    grundgehalt = info.grundentgelt + info.leistungszulage_fix

    urlaubsgeld = info.urlaubsgeld_faktor * grundgehalt
    weihnachtsgeld = info.weihnachtsgeld_faktor * grundgehalt
    t_zug_a = info.t_zug_a_faktor * grundgehalt
    t_zug_b = info.t_zug_b
    t_geld = info.t_geld_faktor * grundgehalt if info.t_geld_aktiv else 0

    jahresgrundgehalt = grundgehalt * 12
    jahressonderzahlungen = urlaubsgeld + weihnachtsgeld + t_zug_a + t_zug_b + t_geld
    jahresbrutto = jahresgrundgehalt + jahressonderzahlungen
    
    nettogrundgehalt = jahresgrundgehalt *  (1 - info.abgaben_standard)
    netto_sonder = jahressonderzahlungen * (1 - info.abgaben_sonderzahlungen)
    jahresnetto = nettogrundgehalt + netto_sonder

    durchschnittsmonatsnetto = jahresnetto / 12

    return {
        "Entgeltgruppe": info.entgeltgruppe,
        "Monatsgesamtgehalt": str(round(grundgehalt, 2))+"€",
        "Tarifliches Zusatzgeld (T-ZUG A)": str(round(t_zug_a,2))+"€",
        "Tariflicher Zusatzbetrag (T-ZUG B)": str(round(t_zug_b,2))+"€",
        "Transformationsgeld (T-Geld)": str(round(t_geld,2))+"€",
        "Urlaubsgeld": str(round(urlaubsgeld,2))+"€",
        "13. Monatseinkommen (Weihnachtsgeld)": str(round(weihnachtsgeld,2))+"€",
        "Jahresbruttogehalt": str(round(jahresbrutto, 2))+"€",
        "Ungefähres Nettojahresgehalt": str(round(jahresnetto, 2))+"€",
        "Ungefähres durschnittliches Monatsnettogehalt": str(round(durchschnittsmonatsnetto, 2))+"€"
    }


tarif_info = TarifInfo(**CONFIG_9b_2025)
ergebnis = berechne_jahresgehalt(tarif_info)

for k, v in ergebnis.items():
    print(f"{k:50}: {v}")


Entgeltgruppe                                     : 9b
Monatsgesamtgehalt                                : 5112.1€
Tarifliches Zusatzgeld (T-ZUG A)                  : 1405.83€
Tariflicher Zusatzbetrag (T-ZUG B)                : 630€
Transformationsgeld (T-Geld)                      : 0€
Urlaubsgeld                                       : 3578.47€
13. Monatseinkommen (Weihnachtsgeld)              : 2811.66€
Jahresbruttogehalt                                : 69771.15€
Ungefähres Nettojahresgehalt                      : 43131.73€
Ungefähres durschnittliches Monatsnettogehalt     : 3594.31€


In [14]:
def gehaltsvergleich(entgeltgruppen: list[TarifInfo]):
    for info in entgeltgruppen:
        ergebnis = berechne_jahresgehalt(info)
        print(f"{info.entgeltgruppe}: {ergebnis['Ungefähres Nettojahresgehalt']}")


In [ ]:
from dataclasses import dataclass

DEFAULTS = {
    "leistungszulage_fix": 271.10,       # Fixe Leistungszulage in € (bis zu 28% des grundentgelts, im Durchschnitt müssen 14% erreicht werden können)
    "urlaubsgeld_faktor": 0.7,           # Ca. 70% eines Monatseinkommens
    "weihnachtsgeld_faktor": 0.55,       # 55% eines Monatseinkommens
    "t_zug_a_faktor": 0.275,             # 27,5% eines Monatseinkommens (T-ZUG A im Juli)
    "t_zug_b": 630,                      # Festbetrag für 2025 (T-ZUG B im Februar)
    "t_geld_faktor": 0.184,              # 18,4% eines Monatseinkommens (T-Geld im Juli)
    "t_geld_aktiv": False,               # Bei Nettoumsatzrendite unter 2,3% kann der Arbeitgeber das T-Geld aussetzen
    "abgaben_standard": 0.38,            # Annahme von ca. 38% Steuern und Sozialabgaben
    "abgaben_sonderzahlungen": 0.395     # Progressiver Steuersatz führt zu höheren Abgaben bei Sonderzahlungen
}


@dataclass
class TarifInfo:
    entgeltgruppe: str
    grundentgelt: float
    jahr: int                             # Jahr der Tarifentgeltgruppe
    stufe: str                            # Stufe A oder B
    leistungszulage_fix: float = 271.10
    urlaubsgeld_faktor: float = 0.7
    weihnachtsgeld_faktor: float = 0.55
    t_zug_a_faktor: float = 0.275
    t_zug_b: float = 630
    t_geld_faktor: float = 0.184
    t_geld_aktiv: bool = False
    abgaben_standard: float = 0.38
    abgaben_sonderzahlungen: float  = 0.395

# Hauptgruppe + Stufe separat angeben
TARIFGRUPPEN = [
    {"entgeltgruppe": "EG09", "stufe": "B", "grundentgelt": 4841, "jahr": 2025},
    {"entgeltgruppe": "EG10", "stufe": "A", "grundentgelt": 5103, "jahr": 2025},
    {"entgeltgruppe": "EG10", "stufe": "B", "grundentgelt": 5362, "jahr": 2025},
    {"entgeltgruppe": "EG11", "stufe": "A", "grundentgelt": 5643, "jahr": 2025},
    {"entgeltgruppe": "EG09", "stufe": "B", "grundentgelt": 4991, "jahr": 2026},
    {"entgeltgruppe": "EG10", "stufe": "A", "grundentgelt": 5261, "jahr": 2026},
    {"entgeltgruppe": "EG10", "stufe": "B", "grundentgelt": 5528, "jahr": 2026},
    {"entgeltgruppe": "EG11", "stufe": "A", "grundentgelt": 5818, "jahr": 2026}
]

def berechne_jahresgehalt(info: TarifInfo):
    grundgehalt = info.grundentgelt + info.leistungszulage_fix

    urlaubsgeld = info.urlaubsgeld_faktor * grundgehalt
    weihnachtsgeld = info.weihnachtsgeld_faktor * grundgehalt
    t_zug_a = info.t_zug_a_faktor * grundgehalt
    t_zug_b = info.t_zug_b
    t_geld = info.t_geld_faktor * grundgehalt if info.t_geld_aktiv else 0

    jahresgrundgehalt = grundgehalt * 12
    jahressonderzahlungen = urlaubsgeld + weihnachtsgeld + t_zug_a + t_zug_b + t_geld
    jahresbrutto = jahresgrundgehalt + jahressonderzahlungen
    
    nettogrundgehalt = jahresgrundgehalt *  (1 - info.abgaben_standard)
    netto_sonder = jahressonderzahlungen * (1 - info.abgaben_sonderzahlungen)
    jahresnetto = nettogrundgehalt + netto_sonder

    durchschnittsmonatsnetto = jahresnetto / 12

    return {
        "Jahr": info.jahr,
        "Entgeltgruppe": info.entgeltgruppe,
        "Stufe": info.stufe,
        "Jahresbruttogehalt": str(round(jahresbrutto, 2))+"€",
        "Ungefähres Nettojahresgehalt": str(round(jahresnetto, 2))+"€",
        "Ungefähres durschnittliches Monatsnettogehalt": str(round(durchschnittsmonatsnetto, 2))+"€"
    }

tarif_infos = [TarifInfo(**DEFAULTS, **tg) for tg in TARIFGRUPPEN]

def gehaltsvergleich(tarif_infos):
    print(f"{'Entgeltgruppe':15} {'Stufe':6} {'Jahr':6} {'Jahresbrutto':20} {'Nettojahresgehalt':25} {'Ø Monatsnetto':20}")
    print("-"*95)
    for info in tarif_infos:
        ergebnis = berechne_jahresgehalt(info)
        print(f"{ergebnis['Entgeltgruppe']:15} "
              f"{ergebnis['Stufe']:6} "
              f"{ergebnis['Jahr']:6} "
              f"{ergebnis['Jahresbruttogehalt']:20} "
              f"{ergebnis['Ungefähres Nettojahresgehalt']:25} "
              f"{ergebnis['Ungefähres durschnittliches Monatsnettogehalt']:20}")

gehaltsvergleich(tarif_infos)


Entgeltgruppe   Stufe  Jahr   Jahresbrutto         Nettojahresgehalt         Ø Monatsnetto       
-----------------------------------------------------------------------------------------------
EG09            B        2025 69771.15€            43131.73€                 3594.31€            
EG10            A        2025 73314.7€             45322.73€                 3776.89€            
EG10            B        2025 76817.68€            47488.65€                 3957.39€            
EG11            B        2025 76817.68€            47488.65€                 3957.39€            
EG09            B        2026 71799.9€             44386.12€                 3698.84€            
EG10            A        2026 75451.65€            46644.03€                 3887.0€             
EG10            B        2026 79062.83€            48876.85€                 4073.07€            
